# Дипломная работа. Задание

Дан файл HR.csv с данными по опросу уровня удовлетворенности сотрудниками работой.

Файл доступен тут -
https://drive.google.com/file/d/1INgo03nal-vwFJe7Lec5vOUtOwfJdUr1/view?usp=sharing

Признаки:
1. satisfaction_level - Уровень удовлетворенности работой
2. Last_evaluation - Время с момента последней оценки в годах
3. number_projects - Количество проектов, выполненных за время работы
4. average_monthly_hours - Среднее количество часов на рабочем месте в месяц
5. time_spend_company - Стаж работы в компании в годах
6. work_accident - Происходили ли несчастные случаи на рабочем месте с сотрудником
7. left - уволился ли сотрудник
8. promotion_last_5years - повышался ли сотрудник за последние пять лет
9. department - отдел в котором работает сотрудник
10. salary - относительный уровень зарплаты

**Требуется выполнить следующее задание:**


### 1. Загрузите файл HR.csv в pandas dataframe 
**5 баллов**

In [ ]:
import pandas as pd
import numpy as np

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns

from pylab import rcParams
rcParams['figure.figsize'] = 11,7


In [ ]:
hr = pd.read_csv('HR.csv')
hr.info()           # проверили на наличие пустых значений и типов данных

### 2. Рассчитайте основные статистики для переменных
(среднее,медиана,мода,мин/макс,сред.отклонение). **10 баллов**

In [ ]:
# на поверхностном уровне проверим мин, макс, средние значения и квантили, а также стандартные отклонения - очевидных выбросов не видно
hr.describe()

In [ ]:
# выводим средние и модальные показатели для столбцов датафрейма
print(f'Средний уровень удовлетворенности работой {hr.satisfaction_level.median()}')
print(f'Время последней оценки в среднем, в годах {hr.last_evaluation.median()}')
print(f'Среднее количество проектов у каждого сотрудника {hr.number_project.median()}')
print(f'Чаще всего у каждого сотрудника {hr.number_project.mode()[0]} проекта(-ов)')
print(f'В среднем каждый сотрудник работает по {hr.average_montly_hours.median()} часов в месяц')
print(f'Обычно сотрудник работает в компании в среднем {hr.time_spend_company.mode()[0]} года')
print()
print(f'Модальное значение несчастных случаев на рабочем месте - {hr.Work_accident.mode()[0]}')
work_accident_count = hr[hr['Work_accident'] == 1]['Work_accident'].count()
print(f'Процент несчастных случаев на рабочем месте - {(work_accident_count*100)/hr.Work_accident.count():.2f} %')
print()
print(f'Модальное значение уволенных - {hr.left.mode()[0]}')
left_count = hr[hr['left'] == 1]['left'].count()
print(f'Процент уволенных сотрудников {(left_count*100)/hr.left.count():.2f} %')
print()
print(f'Модальное значение повышений за последние 5 лет - {hr.promotion_last_5years.mode()[0]}')
promotion_last_5years_count = hr[hr['promotion_last_5years'] == 1]['promotion_last_5years'].count()
print(f'Процент повышенных за последние 5 лет сотрудников {(promotion_last_5years_count*100)/hr.promotion_last_5years.count():.2f} %')
print()
print(f'Чаще всего сотрудники работают в отделе {hr.department.mode()[0]}')
print(f'Чаще всего уровень зарплаты у сотрудника {hr.salary.mode()[0]}')

In [ ]:
# смотрим боксплоты для числовых столбцов датафрейма, чтобы оценить, есть ли в данных выбросы
sns.boxplot(data=hr[['satisfaction_level', 'last_evaluation']]).set_title('Квантили удовлетворенности работой и времени с момента последней оценки')

In [ ]:
sns.boxplot(data=hr[['average_montly_hours']]).set_title('Квантили количества рабочих часов в месяц')

In [ ]:
sns.boxplot(data=hr[['number_project', 'time_spend_company']]).set_title('Квантили количества проектов и времени работы в компании')

# видим, что выбросов в данных нет

In [ ]:
# проверяем категориальные данные, видим, что выбросов и ошибочных данных нет
print(hr.Work_accident.value_counts())
print(hr.left.value_counts())
print(hr.promotion_last_5years.value_counts())
print(hr.department.value_counts())
print(hr.salary.value_counts())

### 3. Рассчитайте и визуализируйте корреляционную матрицу для количественных переменных.  
Определите две самые скоррелированные и две наименее
скоррелированные переменные. **10 баллов**

In [ ]:
# для наглядности превращаем квадратную корреляционную матрицу в треугольную
matrix = np.triu(hr.corr())
sns.heatmap(hr.corr(), cmap='YlGnBu', annot=True, vmin=-1, vmax=1, center=0, mask=matrix)\
.set_title('Корреляционная матрица количественных показателей сотрудников')

# Видим, что два наиболее скоррелированных столбца - average_montly_hours и number_project,
# что логично. Чем больше времени сотрудник проводит на рабочем месте, тем больше проектов успевает сделать.

# Вторая по скоррелированности пара, но с обратной зависимостью - satisfaction_level и left.
# Чем менее сотрудник доволен своей работой, тем вероятнее он окажется уволившимся, и наоборот.

# Наименьшую корреляцию (0,0021) видим между параметрами Work_accident и time_spend_company.
# Стаж работы в компании никак не взаимосвязан с количеством произошедших с сотрудником несчастных случаев.
# Вероятно, условия работы в компании безопасны, 
# и несчастные случаи - это действительно случайность, а не закономерность.

### 4. Рассчитайте сколько сотрудников работает в каждом департаменте.
**5 баллов**

In [ ]:
hr.department.value_counts()
# видим информацию по количеству сотрудников из датафрейма по каждому из департаментов

In [ ]:
# и на всякий случай подсчитаем только тех, кто работает в компании в данный момент
sales_count = hr.loc[(hr.department == 'sales')&(hr.left == 0)]['department'].count()
print(f'В отделе продаж работает {sales_count} сотрудников')
technical_count = hr.loc[(hr.department == 'technical')&(hr.left == 0)]['department'].count()
print(f'В техническом отделе работает {technical_count} сотрудников')
support_count = hr.loc[(hr.department == 'support')&(hr.left == 0)]['department'].count()
print(f'В отделе техподдержки работает {support_count} сотрудников')
IT_count = hr.loc[(hr.department == 'IT')&(hr.left == 0)]['department'].count()
print(f'В айти-отделе работает {IT_count} сотрудников')
product_mng_count = hr.loc[(hr.department == 'product_mng')&(hr.left == 0)]['department'].count()
print(f'В отделе продакт-менеджмента работает {product_mng_count} сотрудников')
marketing_count = hr.loc[(hr.department == 'marketing')&(hr.left == 0)]['department'].count()
print(f'В отделе маркетинга работает {marketing_count} сотрудников')
RandD_count = hr.loc[(hr.department == 'RandD')&(hr.left == 0)]['department'].count()
print(f'В R&D отделе работает {RandD_count} сотрудников')
accounting_count = hr.loc[(hr.department == 'accounting')&(hr.left == 0)]['department'].count()
print(f'В бухгалтерии работает {accounting_count} сотрудников')
hr_count = hr.loc[(hr.department == 'hr')&(hr.left == 0)]['department'].count()
print(f'В отделе кадров работает {hr_count} сотрудников')
management_count = hr.loc[(hr.department == 'management')&(hr.left == 0)]['department'].count()
print(f'В отделе менеджмента работает {management_count} сотрудников')

### 5. Показать распределение сотрудников по зарплатам. 
**5 баллов**

In [ ]:
salary_hist = sns.histplot(x=hr.salary)
salary_hist.set_title('Распределение сотрудников по зарплатам')
salary_hist.set_xlabel('Уровень зарплаты')
salary_hist.set_ylabel('Количество сотрудников')

### 6. Показать распределение сотрудников по зарплатам в каждом департаменте по отдельности
**5 баллов**


In [ ]:
# Строим гистрограмму распределения сотрудников по зарплатам и департаментам 
# в двух видах, чтобы выбрать наиболее наглядный

salary_hist = sns.histplot(y='department', hue='salary', multiple='dodge', data=hr)
salary_hist.axes.set_title('Распределение сотрудников по зарплатам и департаментам')
salary_hist.set_xlabel('Количество сотрудников')
salary_hist.set_ylabel('Департамент')

In [ ]:
# и отдельные гистограммы распределения сотрудников по зарплатам для каждого отдела
salary_hist_2 = sns.displot(x='salary', col='department', data=hr)

salary_hist_2.set_axis_labels('Зарплата', 'Количество сотрудников')
salary_hist_2.fig.suptitle('Распределение сотрудников по департаментам и зарплатам', y=1.03, fontsize=20)

### 7. Проверить гипотезу, что сотрудники с высоким окладом проводят на работе больше времени, чем сотрудники с низким окладом
**10 баллов**

In [ ]:
low_salary = hr.loc[hr.salary == 'low']
print(f'Сотрудники с низким окладом проводят на работе в среднем {low_salary.average_montly_hours.mean():.2f} часов в месяц')

medium_salary = hr.loc[hr.salary == 'medium']
print(f'Сотрудники со средним окладом проводят на работе в среднем {medium_salary.average_montly_hours.mean():.2f} часов в месяц')

high_salary = hr.loc[hr.salary == 'high']
print(f'Сотрудники с высоким окладом проводят на работе в среднем {high_salary.average_montly_hours.mean():.2f} часов в месяц')
print()
print('Видим, что гипотеза не подтверждается:')
print('Сотрудники с высоким окладом проводят на работе меньше времени, чем сотрудники с низким окладом')
print('А сотрудники с низким и средним окладом работают примерно одинаковое количество часов')

### 8. Рассчитать следующие показатели среди уволившихся и не уволившихся сотрудников 
(по отдельности) **10 баллов**

In [ ]:
# разделяем датафреймы на уволившихся и нет

left = hr[hr['left'] == 1]
not_left = hr[hr['left'] == 0]

Доля сотрудников с повышением за последние 5 лет

In [ ]:
def get_percent_by_df_and_column(df, col):
    '''
    Функция высчитывает процент сотрудников, удовлетворяющих категориальному условию.
    df - датафрейм, с которым работаем - целый или подмножество данных.
    col - столбец с категориальными данными, по которым подсчитываем процент единиц.
    На выходе процент.
    '''
    return (df[df[col] == 1][col].count()*100)/(df[col].count())


first_filter = 'promotion_last_5years'
print(f'Среди уволившихся сотрудников доля повышенных за последние пять лет {get_percent_by_df_and_column(left, first_filter):.2f} %')
print(f'Среди работающих сотрудников доля повышенных за последние пять лет {get_percent_by_df_and_column(not_left, first_filter):.2f} %')

Средняя степень удовлетворенности

In [ ]:
def get_mean_by_df_and_column(df, col):
    '''
    Функция высчитывает среднее значение по столбцу col.
    df - датафрейм, с которым работаем - целый или подмножество данных.
    col - столбец с непрерывными данными, по которым подсчитываем процент единиц.
    На выходе среднее.
    '''
    return df[col].mean()

second_filter = 'satisfaction_level'
print(f'Среди уволившихся сотрудников средняя удовлетворенность {get_mean_by_df_and_column(left, second_filter):.2f}')
print(f'Среди работающих сотрудников средняя удовлетворенность {get_mean_by_df_and_column(not_left, second_filter):.2f}')

Среднее количество проектов

In [ ]:
third_filter = 'number_project'
print(f'Среди уволившихся сотрудников среднее количество проектов {get_mean_by_df_and_column(left, third_filter):.2f}')
print(f'Среди уволившихся сотрудников среднее количество проектов {get_mean_by_df_and_column(not_left, third_filter):.2f}')

### 9. Построить модель LDA
Разделить данные на тестовую и обучающую выборки.  
Построить модель LDA, предсказывающую уволился ли
сотрудник на основе имеющихся факторов (кроме department и salary)  
Оценить качество модели на тестовой выборке

**20 баллов**

In [ ]:
# отделяем датафрейм с данными, на основе которых принимается решение, уволится ли человек
lda_df = hr.loc[:, ['satisfaction_level', 'last_evaluation', 'number_project', \
                    'average_montly_hours', 'time_spend_company', 'Work_accident', 'promotion_last_5years']]

# отделяем столбец с результатами - уволится ли человек
lda_left = hr.loc[:, ['left']]

# разделяем данные на тренировочные и тестовые
x_train, x_test, y_train, y_test = train_test_split(lda_df, lda_left, test_size=0.30, random_state=42)

# строим модель линейного дискриминантного анализа (LDA)
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)
lda_result = lda.predict(x_test)

# проверяем модель на тестовых данных и прогнозируемом результате
accuracy_score(y_test, lda_result)

# результат 0,76 близок к единице, значит, модель достаточно хороша и пригодна для использования

In [ ]:
# и на всякий случай проанализируем график остатков. 
# для этого вычтем из реальных тестовых данных те, что получены в результате классификации

leftovers = y_test - lda_result.reshape(-1,1)

# строим гистограмму остатков распределения. столбцов в ней три - остаток может быть только 0, 1 или -1
sns.histplot(x=leftovers.left, bins=3)

# видим нормальное распределение остатков. 
# причем совпадающих результатов в 5-7 раз больше, чем ошибок


### 10. Загрузить jupyter notebook с решение на github и прислать ссылку
5 баллов